# Les cycles d'un graphe

## Motivation : routage dans un réseau

https://en.wikipedia.org/wiki/Spanning_Tree_Protocol


**"broadcast"** : un noeud quelconque du réseau émet une information qui doit être **retransmise** à tous les noeuds du réseau

- N'est possible que si le graphe est connexe !

Proposition d'algorithme : 

- Chaque noeud qui reçoit l'information la retransmet à ses voisins sauf à celui duquel il l'a reçu.

- **Problème** : Si il y a un cycle dans le réseau, l'information va tourner indéfiniment le long du cycle. En réalité, la situation est même bien pire car certain noeud du réseau vont dupliquer l'information ("**broadcast storm**").

## Définition formelle

**Définition** : Un *cycle* d'un graphe $G = (V, E)$ est une liste $C = (c_0, \dots, c_{k-1})$ de sommets telle que
- $k > 2$ : on interdit en particulier les allez-retours
- les $c_i$ sont distincts 
- pour tout $i$ entier, $(c_i, c_{i+1})$ est une arrête de $G$ où $i$ est compris modulo $k$ (c'est-à-dire que $(c_{k-1}, c_0)$ est aussi une arête)

**Note** : Dans le cas des graphes orienté, la notion de cycle ne tient pas compte du sens des arêtes. Si l'on suit  toujours les arêtes dans leurs sens, on parle de *circuit*.

<img src="media/arb-001bis.jpg" width="400" height="400" />

Exemple de cycles dans le graphe précédent: $(1, 2, 13)$ et $(4, 7, 12, 2, 1)$.

Contre-exemple de cycle :
- $(11, 10, 12, 8, 9, 10, 13)$ : passe deux fois par le noeud $10$;
- $(11, 12, 7)$ : $7 - 11$ n'est pas une arête.

À faire:
- remplacer le ficher `graph.py` par le votre après y avoir copier et coller la partie suivant le commentaire
    
     `# Partie sur les cycles dans un graphe à copier dans votre solution`


- Implanter et tester la méthode `is_cycle`

**Note** : pour toutes les méthodes à compléter, on pourra toujours les écrire dans le notebook sous la forme d'une fonction avant de les mettre dans la classe graphe.

In [1]:
from graph import Graph

def is_cycle(G, C):
    return len(C) > 2 and len(set(C)) == len(C) and G.is_path(C) and G.is_edge(C[0], C[-1]) 

G = Graph(vertices=['A', 'B', 'C'], edges=[('A', 'B', 1), ('B', 'C', 1), ('C', 'A', 1)])
assert is_cycle(G, ['A', 'B', 'C'])
assert not is_cycle(G, ['A', 'B'])
assert not is_cycle(G, ['A', 'B', 'A', 'B', 'C'])

G = Graph(vertices=['A', 'B', 'C'], edges=[('A', 'B', 1), ('B', 'C', 1)])
assert not is_cycle(G, ['A', 'B', 'C'])

In [2]:
import graph_examples
examples = graph_examples.Examples(Graph)

G = examples.cours_1_reseau()
assert not G.is_cycle([])
assert not G.is_cycle(["D"])
assert not G.is_cycle(["D", "G"])
assert not G.is_cycle(["D", "H"])
assert G.is_cycle(["D", "H", "C"])
assert G.is_cycle(["A", "G", "F"])
assert G.is_cycle(["D", "H", "G", "B", "A", "F", "E"])
assert not G.is_cycle(["C", "G", "B", "A", "G", "H"])

## Problème de la recherche de cycles dans un graphe

Questions étant donné un graphe $G$:

- le chemin $s$ est-il un cycle de G ? Méthode `G.is_cycle(s)`
- $G$ a-t-il un cycle ?  Méthode `G.is_acyclic()`
- touver un cycle de $G$: Méthode `G.find_cycle()`

**Note** : il n'est pas forcément nécessaire de trouver un cycle pour savoir qu'un graphe contient un cycle.

### Lemme fondamental des cycles dans un graphe

**Lemme 1 :** Soit $G$ un graphe (simple, non orienté). Soient $u, v$ deux sommets de $G$ tel que $u - v$ n'est pas  une arête de $G$. Soit $G'$ le graphe ontenu en ajoutant $u-v$ à $G$. Alors, de deux choses l'une :
- soit $u$ et $v$ sont dans la même composante connexe de $G$, et il y a un cycle dans $G'$ de la forme $(u, \dots, v)$;
- soit $u$ et $v$ ne sont pas dans la même composante connexe de $G$ et alors $G'$ possège une composante connexe de moins que $G$.

À faire : justifier le lemme précédent

1. Si $u$ et $v$ sont dans la même composante connexe alors il existe un chemin de $u$ à $v$ par définition. Donc si l'on ajoute une arête allant de $u$ à $v$ on ferme ce chemin ce qui crée bien un cycle.

2. Si $u$ et $v$ sont dans deux composantes connexes distinctes alors l'ajout d'une arête allant de $u$ à $v$ entraîne la fusion des deux composantes connexes correspondantes. Ainsi on diminue de 1 le nombre de composantes connexes dans le graphe $G'$.

### Nombre d'arcs dans un graphe et cycles

**Proposition 2** : Soit $G = (V;E)$ un graphe. Alors:
1. Si $G$ est connexe, alors $|E| \geq |V| - 1$.
2. Si $G$ est sans cycle, alors $|E|\leq |V| - 1$.

**Preuve** : On va partir du graphe $(V, \emptyset)$ et l'on va rajouter une à une les arêtes de $G$. On remarque qu'au départ le nombre de composantes connexes est $|V|$ et qu'il y aura $|E|$ étapes où l'on rajoute une arête.

1. Comme à chaque étape le nombre de composantes connexes diminue au plus de $1$, il faut au moins $|V| - 1$ étapes pour que le graphe devienne connexe (une composante connexe).
2. Quand on a créé un cycle, rajouter des arêtes en plus le conserve. Si à la fin $G$ est sans cycles, on a donc jamais créé de cycles. Le nombre de composant connexe a donc diminué exactement de $1$ à chaque étape.  

Dans la preuve du point 2, on a en fait prouvé le résultat plus fort suivant:

**Proposition 3** : Un graphe $G=(V;E)$ est **sans cycle** si et seulement si $$|V| = k + |E|$$ où
$k$ est le nombre de composantes connexes.

- Algorithme `is_acyclic(G)` : on fait un parcours de graphe pour compter les composantes connexes


Implanter les méthodes suivantes dans la classe `Graphe`:

- `connected_components`
- `is_connected`
- `is_acyclic`

Pour chacune des méthodes, on donnera ci-dessous où dans le code la complexité dans le cas le pire.

In [3]:
# Version avec pile
def depth_path(G, u):
    """
    INPUT:
    - 'G' - un graphe
    - 'u' - un sommet du graphe
    
    OUTPUT: la liste des sommets `v` de `G`
            tels qu'il existe un chemin de `u` à `v`
    """
    marked = {u} # L'ensemble des sommets déjà rencontrés
    todo   = [u] # Pile des sommets rencontrés
    
    while todo:
        # Invariants:
        # - Si `v` est dans `marked`, alors il y a un chemin de `u` à `v`
        # - Si `v` est dans `marked` et pas dans `todo`
        #   alors tous les voisins de `v` sont dans dans `marked`
        
        # On prend le sommet `v` de la pile et on le dépile
        v = todo.pop()
        # On ajoute directement `v` aux sommets visités
        marked.add(v)
        
        # On empile les voisins `w` de `v`
        for w in G.neighbors(v):
            if w not in marked:
                todo.append(w)
        
    return marked

In [4]:
def connected_components(G):
    res = []
    marked = set()
    for u in G.vertices():
        component = {}
        # On ne regarde pas les composantes connexes déjà trouvées
        if u not in marked:
            component = depth_path(G, u)
            marked.update(component)
            res.append(component)
    return res

La complexité dans le pire des cas de `connected_components` est difficle à établir. Si on considère comme une opération élémentaire un parcours en profondeur, le pire des cas sera atteint quand il n'y a aucune arête donc que chaque sommet est une composante connexe indépendante. Mais dans ce cas là le parcours en profondeur est immédiat car il n'y a rien à parcourir. A l'inverse, nous pourrions donc dire que le pire des cas est atteint si le graphe est connexe donc que le parcours en profondeur est coûteux une fois. 

Il faut alors étudier la complexité du parcours en profondeur `depth_path`. Prenons comme opération élémentaire un tour de boucle. Dans le pire des cas on parcourera tous les sommets et tous leurs voisins. Si chaque sommet est relié à tous les sommets la complexité dans le pire des cas serait donc de $O(|V|^2) = O(|E|^2)$.

In [5]:
G = Graph(vertices=['A', 'B', 'C', 'D'], edges=[('A', 'B', 1), ('B', 'C', 1)])
assert connected_components(G) == [{'A', 'B', 'C'}, {'D'}]
G = Graph(vertices=['A', 'B', 'C', 'D'], edges=[('A', 'B', 1), ('B', 'C', 1), ('C', 'D', 1)])
assert connected_components(G) == [{'A', 'B', 'C', 'D'}]

In [6]:
G = examples.cours_1_reseau()
CCG = G.connected_components()
assert len(CCG) == 1
assert sorted(list(CCG[0])) == ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
assert examples.undirected().connected_components() == [{1, 2, 3}, {4}]
assert examples.disconnected().connected_components() == [{1, 2, 5}, {3, 4}]

In [7]:
def is_connected(G):
    # Un graphe connexe possède une seule composante connexe
    return len(G.connected_components()) == 1

La complexité dans le pire des cas de `is_connected` est clairement celle de `connected_components`.

In [8]:
G = Graph(vertices=['A', 'B', 'C', 'D'], edges=[('A', 'B', 1), ('B', 'C', 1)])
assert not is_connected(G)
G = Graph(vertices=['A', 'B', 'C', 'D'], edges=[('A', 'B', 1), ('B', 'C', 1), ('C', 'D', 1)])
assert is_connected(G)

In [9]:
assert examples.cours_1_reseau().is_connected()
assert not examples.undirected().is_connected()
assert not examples.disconnected().is_connected()
assert examples.tree1().is_connected()

In [10]:
def is_acyclic(G):
    # Un graphe est acyclique si et seulement si |V| = k + |E| avec k le nombre de composantes connexes
    # Dans mon implémentation len(G.edges()) est toujours multiple de 2 car dans un graphe non-orienté
    # les arêtes sont représentées dans les deux sens
    return len(G.vertices()) == (len(G.connected_components()) + (len(G.edges()) // 2))

La complexité dans le pire des cas de `is_acyclic` est la somme des complexités de `Graph.vertices`, `Graph.edges` et `Graph.connected_components`. D'après mon implémentation `Graph.vertices` est en temps constant et `Graph.edges` a une complexité de $2|E|$ dans le cas d'un graphe non-orienté car les arêtes sont représentées dans les deux sens. Donc au total la complexité dans le pire des cas de `is_acyclic` est $O(C + 2|E| + |E|^2)$.

In [11]:
G = Graph(vertices=['A', 'B', 'C'], edges=[('A', 'B', 1), ('B', 'C', 1)])
assert is_acyclic(G)
G = Graph(vertices=['A', 'B', 'C'], edges=[('A', 'B', 1), ('B', 'C', 1), ('C', 'A', 1)])
assert not is_acyclic(G)

In [12]:
assert not examples.cours_1_reseau().is_acyclic()
assert examples.undirected().is_acyclic()
assert examples.disconnected().is_acyclic()
assert examples.lines().is_acyclic()

- Algorithme `find_cycle(G)` : on fait un parcours en profondeur de graphe en partant d'un noeud et en notant pour chaque noeuds traversés comment on l'a atteint. Si l'on atteint deux fois un noeuds, on a trouvé un cycle.


Implanter la méthode `find_cycle(G)` et donner sa complexité.

In [13]:
# Version avec pile
def depth_path_cycle(G, u):
    """
    INPUT:
    - 'G' - un graphe
    - 'u' - un sommet du graphe
    
    OUTPUT: la liste des sommets `v` de `G`
            tels qu'il existe un chemin de `u` à `v`
    """
    todo = [(None, u)] # Pile des sommets rencontrés
    path = [] # Chemin actuel depuis la racine 
    
    while todo:
        # Invariants:
        # - Si `v` est dans `marked`, alors il y a un chemin de `u` à `v`
        # - Si `v` est dans `marked` et pas dans `todo`
        #   alors tous les voisins de `v` sont dans dans `marked`
        
        # On prend le sommet `v` de la pile et on le dépile
        (_, v) = todo.pop()
        
        # Nombre d'éléments empilés
        size = len(todo)
        
        # On empile les voisins `w` de `v`
        for w in G.neighbors(v):
            # Si on détecte un cyle alors on construit son chemin
            if w in path[:-1]:
                cycle = [w, v]
                t = path.pop()
                while t != w:
                    cycle.append(t)
                    t = path.pop()
                # On stoppe la recherche
                return cycle
            # Sinon si le voisin n'est pas le dernier sommet du chemin alors on l'empile
            elif w not in path[-1:]:
                todo.append((v, w))
        
        # Si on a trouvé des voisins pour `v`
        # Alors on ajoute le sommet `v` au chemin actuel
        if size < len(todo):
            path.append(v)
            
        # Sinon si la pile n'est pas vide
        # Alors on réduit le chemin actuel
        elif 0 < size:
            (t, _) = todo[-1]
            # On ne sait pas combien de sommets retirer
            while t != path[-1]:
                path.pop()
            
        print(todo, path)
        
    return None

In [14]:
G = Graph(vertices=['A', 'B', 'C'], edges=[('A', 'B', 1), ('B', 'C', 1)])
depth_path_cycle(G, 'A')

[('A', 'B')] ['A']
[('B', 'C')] ['A', 'B']
[] ['A', 'B']


In [15]:
G = Graph(vertices=['A', 'B', 'C'], edges=[('A', 'B', 1), ('B', 'C', 1), ('C', 'A', 1)])
depth_path_cycle(G, 'A')

[('A', 'B'), ('A', 'C')] ['A']
[('A', 'B'), ('C', 'B')] ['A', 'C']


['A', 'B', 'C']

In [16]:
G = Graph(vertices=['A', 'B', 'C', 'D', 'E'], edges=[
    ('A', 'B', 1), ('A', 'C', 1), ('C', 'D', 1), ('C', 'E', 1)])
depth_path_cycle(G, 'A')

[('A', 'B'), ('A', 'C')] ['A']
[('A', 'B'), ('C', 'D'), ('C', 'E')] ['A', 'C']
[('A', 'B'), ('C', 'D')] ['A', 'C']
[('A', 'B')] ['A']
[] ['A']


In [17]:
G = Graph(vertices=['A', 'B', 'C', 'D', 'E'], edges=[
    ('A', 'B', 1), ('A', 'C', 1), ('C', 'D', 1), ('C', 'E', 1), ('B', 'D', 1)])
depth_path_cycle(G, 'A')

[('A', 'B'), ('A', 'C')] ['A']
[('A', 'B'), ('C', 'D'), ('C', 'E')] ['A', 'C']
[('A', 'B'), ('C', 'D')] ['A', 'C']
[('A', 'B'), ('D', 'B')] ['A', 'C', 'D']


['A', 'B', 'D', 'C']

In [18]:
def find_cycle(G):
    cycle = None
    # Tant qu'on ne trouve pas de cycle on tente de faire
    # une recherche depuis chaque sommet
    for u in G.vertices():
        cycle = depth_path_cycle(G, u)
        if cycle is not None:
            break
    return cycle

Le pire des cas de `find_cycle` est atteint lorsque l'on a itéré sur tous les sommets sans trouver de cycle. Dans ce cas la complexité correspond à la complexité de `depth_path_cycle` multipliée par le nombre $|V|$. Bien que le parcours en profondeur soit modifié pour détecter les cycles, cela ne modifie pas sa complexité dans le pire des cas qui restera de $|V|^2$. En revanche, il utilisera davantage de mémoire car on a besoin de stocker davantage d'informations. Au total, la complexité dans le pire des cas de `find_cycle` est cubique en $O(|V|^3)$  

In [19]:
G = Graph(vertices=['A', 'B', 'C'], edges=[('A', 'B', 1), ('B', 'C', 1)])
find_cycle(G)

[('A', 'B')] ['A']
[('B', 'C')] ['A', 'B']
[] ['A', 'B']
[('B', 'A'), ('B', 'C')] ['B']
[('B', 'A')] ['B']
[] ['B']
[('C', 'B')] ['C']
[('B', 'A')] ['C', 'B']
[] ['C', 'B']


In [20]:
G = Graph(vertices=['A', 'B', 'C'], edges=[('A', 'B', 1), ('B', 'C', 1), ('C', 'A', 1)])
find_cycle(G)

[('A', 'B'), ('A', 'C')] ['A']
[('A', 'B'), ('C', 'B')] ['A', 'C']


['A', 'B', 'C']

In [21]:
G = examples.cours_1_reseau()
assert G.is_cycle(G.find_cycle())
G = examples.G2()
assert G.is_cycle(G.find_cycle())
assert examples.undirected().find_cycle() is None
assert examples.disconnected().find_cycle() is None
for t in examples.trees():
    assert(t.find_cycle() is None)
for G in examples.cyclic():
    assert(G.is_cycle(G.find_cycle()))

## Notion d'arbre

**Définition** : Un *arbre* est un graphe **connexe** et **sans cycle** (on dit aussi **acyclique**)

Dans un arbre, on a pas de problème pour le routage:

**Proposition 4.** $G$ est un arbre si et seulement si, pour tout couple de sommets $s$ et $t$, il existe un **unique chemin simple** (qui ne repasse jamais deux fois par le même sommet) de $s$ à $t$.

**Proposition 5. (Caractérisations d'un Arbre)** :
Pour un graphe $T$ à $n$ sommets, il y a équivalence entre les propriétés :

1. $T$ est un arbre
- $T$ est un graphe connexe à $n-1$ arêtes
- $T$ est acyclique à $n-1$ arêtes
- $T$ est connexe, et la suppression de toute arête le déconnecte
- $T$ est acyclique et l'ajout de toute arête le rend cyclique.


**Preuve** : 
- $1.\Rightarrow 2.$ et $1.\Rightarrow 3.$ sont des conséquence immédiate de la proposition 2
- $2.\Rightarrow 1.$ et $3.\Rightarrow 1.$ sont des conséquence de la proposition 3.

À faire : justifier ci-dessous les équivalences des points 4. et 5. 

1. => 4. car un arbre est un graphe connecté par définition où il existe un unique chemin simple entre tout couple de sommets $(s, t)$ d'après la proposition 4.

4. => 1. Supposons que $T'$ ne soit pas un arbre mais que ce soit un graphe connexe dont la suppression de toute arête le déconnecte. Cela signifie que $T'$ n'est pas un graphe à la fois connexe et acyclique. Or, il est déjà connexe. Mais il doit aussi être nécessairement acyclique car autrement la suppression d'au moins une arête permettrait de ne pas le déconnecter. Donc par l'absure $T' = T$ qui est un arbre.

1. => 5. car un arbre est un graphe connexe et acyclique par définition où il existe un unique chemin simple entre tout couple de sommets $(s, t)$ d'après la proposition 4. Donc si on ajoute une arête entre deux sommets $(s, t)$ comme il existe déjà un chemin entre $(s, t)$ on crée un cycle.

5. => 1. Supposons que $T'$ ne soit pas un arbre mais que ce soit un graphe acyclique dont l'ajout de toute arête le rend cyclique. Comme ce n'est pas un arbre il ne peut pas être à la fois connexe et acyclique. Il est déjà acyclique dont il ne doit pas être connexe. Or, pour que l'ajout de toute arête le rende cyclique il faut nécessairement qu'il soit connexe. En effet, il faut déjà qu'un chemin entre les sommets $(s, t)$ existe pour que l'ajout de l'arête $(s, t)$ crée un cycle. Donc par l'absurde $T' = T$ qui est un arbre.

Implanter la méthode `G.is_tree()` et donner sa complexité.

In [22]:
def is_tree(G):
    # Un arbre est un graphe connexe et acyclique
    return G.is_connected() and len(G.vertices()) == (len(G.edges()) // 2) + 1

In [23]:
G = Graph(vertices=['A', 'B', 'C'], edges=[('A', 'B', 1), ('B', 'C', 1)])
assert is_tree(G)
G = Graph(vertices=['A', 'B', 'C'], edges=[('A', 'B', 1), ('B', 'C', 1), ('C', 'A', 1)])
assert not is_tree(G)

In [24]:
assert not examples.cours_1_reseau().is_tree()
assert not examples.undirected().is_tree()
assert not examples.disconnected().is_tree()
assert examples.tree1().is_tree()
assert not examples.lines().is_tree()

La complexité dans le pire des cas de `is_tree` est identique à celle de `is_acyclic` soit $O(C + 2|E| + |E|^2)$.